In [1]:
import os

# Check if the notebook is running on Colab
if 'COLAB_GPU' in os.environ:
    # This block will run only in Google Colab
    IN_COLAB = True
    print("Running on Google Colab. Cloning the repository.")
    !git clone https://github.com/pedro15sousa/energy-based-models-compression.git
    %cd energy-based-models-compression/notebooks
else: 
    # This block will run if not in Google Colab
    IN_COLAB = False
    print("Not running on Google Colab. Assuming local environment.")

Not running on Google Colab. Assuming local environment.


In [2]:
import sys
sys.path.append('..')  # This adds the parent directory (main_folder) to the Python path

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import MNIST
import torch.utils.data as data

# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning
    import pytorch_lightning as pl
# Callbacks
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
# Pytorch Summary
try:
    from torchsummary import summary
except ModuleNotFoundError:
    !pip install --quiet torchsummary
    from torchsummary import summary

import numpy as np
import pandas as pd
import json

## Imports for plotting
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from matplotlib import cm
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
from mpl_toolkits.mplot3d.axes3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()

from metrics.classifier import VGG
from EBM import DeepEnergyModel
from callbacks import InceptionScoreCallback, \
    FIDCallback, SamplerCallback, OutlierCallback, \
    GenerateImagesCallback

import shutil
if IN_COLAB:
    from google.colab import files

# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10)
DATASET_PATH = "../data"
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "../saved_models"

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device: ", device)

/var/folders/pl/gjvtfzbj4691nr_0pszjdspw0000gn/T/ipykernel_27356/932283081.py:33: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export


Device: cpu
Device:  cpu


In [4]:
# Transformations applied on each image => make them a tensor and normalize between -1 and 1
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))
                               ])

# Loading the training dataset. We need to split it into a training and validation part
train_set = MNIST(root=DATASET_PATH, train=True, transform=transform, download=True)

# Loading the test set
test_set = MNIST(root=DATASET_PATH, train=False, transform=transform, download=True)

# We define a set of data loaders that we can use for various purposes later.
# Note that for actually training a model, we will use different data loaders
# with a lower batch size.
train_loader = data.DataLoader(train_set, batch_size=64, shuffle=True,  drop_last=True,  num_workers=2, pin_memory=True)
test_loader  = data.DataLoader(test_set,  batch_size=128, shuffle=False, drop_last=False, num_workers=2)

In [5]:
if os.path.exists('../saved_models/mnist-classifier-1 (1).pth'):
    # Load the best model
    mnist_classifier = VGG()

    if device == 'cuda':
        mnist_classifier.load_state_dict(torch.load('../saved_models/mnist-classifier-1 (1).pth'))
    else:
        mnist_classifier.load_state_dict(torch.load('../saved_models/mnist-classifier-1 (1).pth', map_location=torch.device('cpu')))

    mnist_classifier.to(device)
    print("Model already exists and loaded.")
    summary(mnist_classifier, input_size=(1, 28, 28))
else:
    print("Classifier not found in saved_models. Please run the classifier notebook first.")

Model already exists and loaded.
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 28, 28]           1,280
              ReLU-2          [-1, 128, 28, 28]               0
            Conv2d-3          [-1, 128, 28, 28]         147,584
              ReLU-4          [-1, 128, 28, 28]               0
            Conv2d-5          [-1, 128, 28, 28]         147,584
              ReLU-6          [-1, 128, 28, 28]               0
         MaxPool2d-7          [-1, 128, 14, 14]               0
            Conv2d-8          [-1, 256, 14, 14]         295,168
              ReLU-9          [-1, 256, 14, 14]               0
           Conv2d-10          [-1, 256, 14, 14]         590,080
             ReLU-11          [-1, 256, 14, 14]               0
           Conv2d-12          [-1, 256, 14, 14]         590,080
             ReLU-13          [-1, 256, 14, 14]               0
      

In [6]:
class DownloadCheckpointCallback(pl.Callback):
    def __init__(self, download_dir, stru, ratio, every_n_epochs=5):
        """
        Args:
            download_interval (int): Number of epochs between downloads.
            download_dir (str): Directory to download checkpoints to.
        """
        self.every_n_epochs = every_n_epochs
        self.download_dir = download_dir
        self.stru = stru
        self.ration = ratio

    def on_train_epoch_end(self, trainer, pl_module, unused=None):
        epoch = trainer.current_epoch
        if epoch % self.every_n_epochs == 0:
            if IN_COLAB:
                shutil.make_archive(f'MNIST_{self.ratio}_{self.stru}_{epoch}', 'zip', self.download_dir)
                files.download(f'MNIST_{self.ratio}_{self.stru}_{epoch}.zip')
            else:
                return

In [7]:
def save_scores(trainer, default_root_dir):
    is_callback = [cb for cb in trainer.callbacks if isinstance(cb, InceptionScoreCallback)][0]
    epoch_is_scores = is_callback.scores
    is_path = os.path.join(default_root_dir, "epoch_is_scores.json")

    with open(is_path, 'w') as f:
        json.dump(epoch_is_scores, f)

    fid_callback = [cb for cb in trainer.callbacks if isinstance(cb, FIDCallback)][0]
    epoch_fid_scores = fid_callback.scores
    fid_path = os.path.join(default_root_dir, "epoch_fid_scores.json")

    with open(fid_path, 'w') as f:
        json.dump(epoch_fid_scores, f)

In [11]:
def train_model(stru, ratio, **kwargs):
    perc = str(ratio * 100)
    default_root_dir = os.path.join(CHECKPOINT_PATH, f"MNIST/{perc}/{stru}")
    # Create a PyTorch Lightning trainer with the generation callback
    trainer = pl.Trainer(default_root_dir=default_root_dir,
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=10,
                         gradient_clip_val=0.1,
                         callbacks=[ModelCheckpoint(dirpath=default_root_dir, filename='MNIST_pruned_{ratio}_{stru}-{epoch:02d}', save_top_k=-1, every_n_epochs=1),
                                    GenerateImagesCallback(every_n_epochs=3),
                                    SamplerCallback(every_n_epochs=3),
                                    OutlierCallback(),
                                    LearningRateMonitor("epoch"),
                                    InceptionScoreCallback(mnist_classifier),
                                    FIDCallback(mnist_classifier),
                                    DownloadCheckpointCallback(default_root_dir, {stru}, {perc}, every_n_epochs=3)
                                   ])

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, f"MNIST.ckpt")
    if os.path.isfile(pretrained_filename):
        print("Found pretrained model, loading...")
        model = DeepEnergyModel.load_from_checkpoint(pretrained_filename)
    else:
        print("No pretrained model found. Start training from scratch...")
        pl.seed_everything(42)
        model = DeepEnergyModel(**kwargs)

    # Apply pruning before tuning
    if stru == 'structured':
        print(f"Applying structured pruning with a ratio of {ratio}.")
        model.apply_structured_pruning(amount=ratio)
    else:
        print(f"Applying unstructured pruning with a ratio of {ratio}.")
        model.apply_unstructured_pruning(amount=ratio)  # Example for unstructured pruning
    
    # Print pruning statistics
    model.print_pruning_stats()

    trainer.fit(model, train_loader, test_loader)

    model = DeepEnergyModel.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
    save_scores(trainer, default_root_dir)
        
    # No testing as we are more interested in other properties
    return model

In [9]:
pretrained_filename = os.path.join(CHECKPOINT_PATH, "MNIST.ckpt")
model = DeepEnergyModel.load_from_checkpoint(pretrained_filename)
# model = DeepEnergyModel.load_from_checkpoint(pretrained_filename)
summary(model, input_size=(1, 28, 28))
# pl.seed_everything(43)

Lightning automatically upgraded your loaded checkpoint from v1.0.2 to v2.1.3. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../saved_models/MNIST.ckpt`


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 16, 16]             416
             Swish-2           [-1, 16, 16, 16]               0
            Conv2d-3             [-1, 32, 8, 8]           4,640
             Swish-4             [-1, 32, 8, 8]               0
            Conv2d-5             [-1, 64, 4, 4]          18,496
             Swish-6             [-1, 64, 4, 4]               0
            Conv2d-7             [-1, 64, 2, 2]          36,928
             Swish-8             [-1, 64, 2, 2]               0
           Flatten-9                  [-1, 256]               0
           Linear-10                   [-1, 64]          16,448
            Swish-11                   [-1, 64]               0
           Linear-12                    [-1, 1]              65
         CNNModel-13                       [-1]               0
Total params: 76,993
Trainable params: 

# Pre-Training

## 50%

### Unstructured

In [13]:
if IN_COLAB:
    %reload_ext tensorboard
    %tensorboard --logdir saved_models/MNIST/lightning_logs

In [12]:
# MNIST dataset images are 28x28 pixels in size and are black and white, so only have one channel
un_50_model = train_model("unstructured",
                    ratio=0.5,
                    img_shape=(1,28,28),
                    batch_size=train_loader.batch_size,
                    lr=1e-4,
                    beta1=0.0)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/pedrosousa/anaconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
Lightning automatically upgraded your loaded checkpoint from v1.0.2 to v2.1.3. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../saved_models/MNIST.ckpt`


Found pretrained model, loading...
Applying unstructured pruning with a ratio of 0.5.
Total parameters: 76993
Pruned parameters: 30152


/Users/pedrosousa/anaconda3/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /Users/pedrosousa/Documents/Cambridge/Principles of ML Systems/energy-based-models-compression/saved_models/MNIST/50.0/unstructured exists and is not empty.

  | Name | Type     | Params | In sizes       | Out sizes
---------------------------------------------------------------
0 | cnn  | CNNModel | 77.0 K | [1, 1, 28, 28] | [1]      
---------------------------------------------------------------
77.0 K    Trainable params
0         Non-trainable params
77.0 K    Total params
0.308     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/pedrosousa/anaconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
/Users/pedrosousa/anaconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Training: |          | 0/? [00:00<?, ?it/s]


KeyboardInterrupt



### Structured

In [27]:
if IN_COLAB:
    %reload_ext tensorboard
    %tensorboard --logdir saved_models/MNIST/lightning_logs

In [ ]:
# MNIST dataset images are 28x28 pixels in size and are black and white, so only have one channel
stru_50_model = train_model("structured",
                    ratio=0.5,
                    img_shape=(1,28,28),
                    batch_size=train_loader.batch_size,
                    lr=1e-4,
                    beta1=0.0)

## 25%

### Unstructured

In [38]:
if IN_COLAB:
    %reload_ext tensorboard
    %tensorboard --logdir saved_models/MNIST/lightning_logs

In [ ]:
# MNIST dataset images are 28x28 pixels in size and are black and white, so only have one channel
un_25_model = train_model("unstructured",
                    ratio=0.25,
                    img_shape=(1,28,28),
                    batch_size=train_loader.batch_size,
                    lr=1e-4,
                    beta1=0.0)

### Structured

In [40]:
if IN_COLAB:
    %reload_ext tensorboard
    %tensorboard --logdir saved_models/MNIST/lightning_logs

In [ ]:
# MNIST dataset images are 28x28 pixels in size and are black and white, so only have one channel
stru_25_model = train_model("structured",
                    ratio=0.25,
                    img_shape=(1,28,28),
                    batch_size=train_loader.batch_size,
                    lr=1e-4,
                    beta1=0.0)

## 10%

### Unstructured

In [41]:
if IN_COLAB:
    %reload_ext tensorboard
    %tensorboard --logdir saved_models/MNIST/lightning_logs

In [ ]:
# MNIST dataset images are 28x28 pixels in size and are black and white, so only have one channel
un_10_model = train_model("unstructured",
                    ratio=0.1,
                    img_shape=(1,28,28),
                    batch_size=train_loader.batch_size,
                    lr=1e-4,
                    beta1=0.0)

### Structured

In [ ]:
if IN_COLAB:
    %reload_ext tensorboard
    %tensorboard --logdir saved_models/MNIST/lightning_logs

In [ ]:
# MNIST dataset images are 28x28 pixels in size and are black and white, so only have one channel
stru_10_model = train_model("structured",
                    ratio=0.1,
                    img_shape=(1,28,28),
                    batch_size=train_loader.batch_size,
                    lr=1e-4,
                    beta1=0.0)

## 75%

### Unstructured

In [ ]:
if IN_COLAB:
    %reload_ext tensorboard
    %tensorboard --logdir saved_models/MNIST/lightning_logs

In [ ]:
# MNIST dataset images are 28x28 pixels in size and are black and white, so only have one channel
un_75_model = train_model("unstructured",
                    ratio=0.75,
                    img_shape=(1,28,28),
                    batch_size=train_loader.batch_size,
                    lr=1e-4,
                    beta1=0.0)

### Structured

In [ ]:
if IN_COLAB:
    %reload_ext tensorboard
    %tensorboard --logdir saved_models/MNIST/lightning_logs

In [ ]:
# MNIST dataset images are 28x28 pixels in size and are black and white, so only have one channel
stru_75_model = train_model("structured",
                    ratio=0.75,
                    img_shape=(1,28,28),
                    batch_size=train_loader.batch_size,
                    lr=1e-4,
                    beta1=0.0)

## 90%

### Unstructured

In [ ]:
if IN_COLAB:
    %reload_ext tensorboard
    %tensorboard --logdir saved_models/MNIST/lightning_logs

In [ ]:
# MNIST dataset images are 28x28 pixels in size and are black and white, so only have one channel
un_90_model = train_model("unstructured",
                    ratio=0.90,
                    img_shape=(1,28,28),
                    batch_size=train_loader.batch_size,
                    lr=1e-4,
                    beta1=0.0)

### Structured

In [ ]:
if IN_COLAB:
    %reload_ext tensorboard
    %tensorboard --logdir saved_models/MNIST/lightning_logs

In [ ]:
# MNIST dataset images are 28x28 pixels in size and are black and white, so only have one channel
stru_90_model = train_model("structured",
                    ratio=0.90,
                    img_shape=(1,28,28),
                    batch_size=train_loader.batch_size,
                    lr=1e-4,
                    beta1=0.0)